In [1]:
import numpy as np
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
from performStockAnalysis.main import select_kode_to_model, develop_models_for_selected_kode, make_forecasts_using_the_developed_models

In [ ]:
selected_kode = select_kode_to_model()

In [ ]:
develop_models_for_selected_kode(selected_kode)

In [ ]:
event_date = '20250907'
make_forecasts_using_the_developed_models(10, event_date, 0.7)

2025-09-07 21:54:45 - INFO - Starting the Process of 10 Days Forecasting
2025-09-07 21:54:45 - INFO - Loading the data from database/modelPerformances/modelPerformance-10dd-20250907.csv
2025-09-07 21:54:45 - INFO - Select stock with the model performance on testing data that is greater than 0.7
2025-09-07 21:54:45 - INFO - Selected a total of 21 stocks that exceed the minimum model performance
2025-09-07 21:54:45 - INFO - Prepare all stock data to be used for forecasting
2025-09-07 21:54:45 - INFO - Starting Data Preparation Pipeline for 21 Tickers
2025-09-07 21:54:45 - INFO - Downloading stock data for ticker BELL.JK
2025-09-07 21:54:45 - INFO - Generating technical indicators as features
2025-09-07 21:54:45 - INFO - Processing the data into a format that is compatible for generating the stock indicators
2025-09-07 21:54:46 - INFO - Generating stock indicators of price trends
2025-09-07 21:54:47 - INFO - Generating stock indicators of price channels
2025-09-07 21:54:52 - INFO - Genera

In [ ]:
event_date = '20250907'
make_forecasts_using_the_developed_models(15, event_date, 0.6)

In [5]:
def analysis_for_decision_making(forecast_dd: int, development_date: str, train_gini: float = 0.5, test_gini: float = 0.45, train_precision_up: float = 0.65, test_precision_up: float = 0.6):
    model_performance = pd.read_csv(f'database/modelPerformances/modelPerformance-{forecast_dd}dd-{development_date}.csv')
    forecast_result = pd.read_csv(f'database/forecastedStocks/forecast-{forecast_dd}dd-{development_date}.csv') \
                            .drop(columns=['Date'])

    filter_data = np.all((
                            model_performance['Train - Gini'] >= train_gini,
                            model_performance['Test - Gini'] >= test_gini, 
                            model_performance['Train - Precision Up Trend'] >= train_precision_up,
                            model_performance['Test - Precision Up Trend'] >= test_precision_up, 
                            ), 
                         axis=0
                        )
    
    data_for_decision = pd.merge(
                                    model_performance.loc[
                                                            filter_data,
                                                            :
                                                        ],
                                    forecast_result,
                                    on='Kode',
                                    how='inner'
                                ) \
                                .sort_values(f'Forecast - Upcoming {forecast_dd} Days Trend', ascending=False) \
                                .reset_index(drop=True)

    return data_for_decision

In [6]:
analysis_for_decision_making(10, '20250907')

,Kode,Train - Accuracy,Train - Precision Up Trend,Train - Precision Down Trend,Train - Recall Up Trend,Train - Recall Down Trend,Train - Gini,Test - Accuracy,Test - Precision Up Trend,Test - Precision Down Trend,Test - Recall Up Trend,Test - Recall Down Trend,Test - Gini,Forecast - Upcoming 10 Days Trend
0,GRPH,0.983498,0.983498,0.000000,1.000000,0.000000,0.912752,0.933333,0.933333,0.00,1.000000,0.0,1.000000,0.847759
1,SLIS,0.752627,0.860068,0.710875,0.536170,0.928943,0.640743,0.966667,0.965517,1.00,1.000000,0.5,1.000000,0.433349
2,ADCP,0.828534,0.869198,0.762069,0.856549,0.780919,0.821573,0.966667,0.966667,0.00,1.000000,0.0,1.000000,0.336447
3,KIJA,0.944604,0.961123,0.931507,0.917526,0.967972,0.978101,0.700000,1.000000,0.10,0.689655,1.0,1.000000,0.281480
4,MLIA,0.938873,0.952070,0.928571,0.912317,0.961268,0.974055,0.566667,1.000000,0.35,0.434783,1.0,0.950311,0.242572
5,BELL,0.881827,0.918699,0.860502,0.792056,0.948187,0.909786,0.900000,1.000000,0.40,0.892857,1.0,1.000000,0.144018
6,BTEK,0.802974,0.786982,0.830000,0.886667,0.697479,0.766443,0.466667,1.000000,0.20,0.384615,1.0,0.971154,0.122032
7,MEDS,0.978659,0.987069,0.958333,0.982833,0.968421,0.995911,0.966667,0.966667,0.00,1.000000,0.0,1.000000,0.060613
